In [ ]:
project = 'lcvae'

In [ ]:
n_samples = 3285

In [ ]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import nn, optim
torch.manual_seed(3409)

In [ ]:
import numpy as np
np.random.seed(0)
import pandas as pd
from pandas.core.frame import DataFrame

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import time
timestr = time.strftime("%Y%m%d-%H%M%S")

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
train_balanced = pd.read_csv("/content/drive/MyDrive/20221108/dataset/train_balanced.csv")

In [ ]:
import sys
sys.path.insert(0,'/content/drive/MyDrive/20221108')

In [ ]:
import databuild

# Load model

In [ ]:
class Autoencoder(nn.Module):
  
    def __init__(self, D_in, H1, H2, H3, latent_dim): 
      
        super(Autoencoder,self).__init__()
        
        self.fc21 = nn.Linear(latent_dim, latent_dim)
        self.fc22 = nn.Linear(latent_dim, latent_dim)
        
        self.encoder = nn.Sequential(
                nn.Linear(D_in, H1), nn.BatchNorm1d(H1), nn.ReLU(),
                nn.Linear(H1, H2),  nn.BatchNorm1d(H2), nn.ReLU(),
                nn.Linear(H2, H3),  nn.BatchNorm1d(H3), nn.ReLU(), 
                nn.Linear(H3, latent_dim), nn.BatchNorm1d(latent_dim), nn.ReLU(),
                nn.Linear(latent_dim, latent_dim), nn.BatchNorm1d(latent_dim), nn.ReLU(),
                ) 

        self.decoder = nn.Sequential(
                nn.Linear(latent_dim, latent_dim), nn.BatchNorm1d(latent_dim), nn.ReLU(), 
                nn.Linear(latent_dim, H3), nn.BatchNorm1d(H3), nn.ReLU(), 
                nn.Linear(H3, H2),  nn.BatchNorm1d(H2),  nn.ReLU(), 
                nn.Linear(H2, H1),  nn.BatchNorm1d(H1),  nn.ReLU(), 
                nn.Linear(H1, D_in), nn.BatchNorm1d(D_in), nn.ReLU(),
                )
        
        self.classifier = nn.Sequential(
                nn.Linear(D_in, D_in), nn.ReLU(),
                nn.Linear(D_in, D_in), nn.ReLU(),
                nn.Linear(D_in, 2),   nn.Sigmoid(),
                )
    

    def forward(self, x): 
        latent_fea = self.encoder(x)
        mu, logvar = self.compute_r1_r2(latent_fea)
        z = self.reparameterize(mu, logvar)
        latent_label = self.classifier(z) 
        generated_data = self.decoder(z)
        return  mu, logvar, latent_fea, z, generated_data, latent_label
        # x (encoder) latent_fea
        # latent_fea (fit) z, latent_label
        # z (decoder) generated data, output_label
      
    def compute_r1_r2(self, latent_fea):
        r1 = self.fc21(latent_fea)
        r2 = self.fc22(latent_fea)
        return r1, r2

    def reparameterize(self, mu, logvar):
        if self.training:
            std = logvar.mul(0.5).exp_()
            eps = Variable(std.data.new(std.size()).normal_())
            return eps.mul(std).add_(mu)
        else:
            return mu

In [ ]:
D_in = 4
H1 = 4
H2 = 4
H3 = 4
latent_dim = 4
model = Autoencoder(D_in, H1, H2, H3, latent_dim)

model.load_state_dict(torch.load(f"/content/drive/MyDrive/20221108/saved_model/{project}.pth", map_location='cpu'))
model.eval()

Autoencoder(
  (fc21): Linear(in_features=4, out_features=4, bias=True)
  (fc22): Linear(in_features=4, out_features=4, bias=True)
  (encoder): Sequential(
    (0): Linear(in_features=4, out_features=4, bias=True)
    (1): BatchNorm1d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=4, out_features=4, bias=True)
    (4): BatchNorm1d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Linear(in_features=4, out_features=4, bias=True)
    (7): BatchNorm1d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU()
    (9): Linear(in_features=4, out_features=4, bias=True)
    (10): BatchNorm1d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU()
    (12): Linear(in_features=4, out_features=4, bias=True)
    (13): BatchNorm1d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (14): ReLU()
  )
  (decoder): Sequential(


In [ ]:
learning_rate = 1e-3
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Generate

In [ ]:
def gaussian(dimension, n_sample):
  
    # Set mean vector. 
    m = np.array([1, 1, 1 ,1]).reshape(4, 1)
    # Set covariance function. 
    K_0 = np.array([[2, 1, 1, 1],
                    [1, 2, 1, 1],
                    [1, 1, 2, 1],
                    [1, 1, 1, 2]])
    # Eigenvalues covariance function.
    np.linalg.eigvals(K_0)
    # Define epsilon.
    epsilon = 0.0001
    # Add small pertturbation. 
    K = K_0 + epsilon*np.identity(dimension)
    # Cholesky decomposition.
    L = np.linalg.cholesky(K)
    np.dot(L, np.transpose(L))

    u = np.random.normal(loc = 0, scale = 1, size = dimension * n_sample).reshape(dimension, n_sample)
    x = m + np.dot(L, u)
    z = np.random.multivariate_normal(mean = m.reshape(dimension,), cov = K, size = n_sample)
    
    return z

In [ ]:
@torch.no_grad()
def generate_fake(model, z):
    pred = model.decoder(z).cpu().numpy()
    return pred

## non-coalescence

In [ ]:
train_0 = train_balanced[train_balanced.iloc[: , -1] == 0]
train_0_X = train_0.iloc[:, 0:-1]

In [ ]:
generatedata0 = databuild.DataBuilder(train_0_X)
generateloader0 = databuild.DataLoader(dataset = generatedata0, batch_size = 219)

with torch.no_grad():

  for batch_idx, data in enumerate(generateloader0): 
    data = data.to(device)
    optimizer.zero_grad()
    mu, logvar, latent_fea, z, generated_data, latent_label = model(data)

input_data_tensor = torch.tensor(train_0_X.values.astype(np.float32)) 

sigma = torch.exp(logvar/2)

# sample z from q
no_samples = n_samples
q = torch.distributions.Normal(mu.mean(axis=0), sigma.mean(axis=0))
noise = gaussian(4, n_samples)
z = q.rsample(sample_shape=torch.Size([no_samples])) + 0.5 * torch.tensor(noise).float()

generated_data = generate_fake(model, z)
cols = ["chamber_size","top","bottom","dt"]
df_fake_0 = pd.DataFrame(generated_data, columns = cols) 

latent_fea_tensor = latent_fea.clone().detach().requires_grad_(True)
latent_fea = latent_fea_tensor.detach().numpy()
latent_fea_0 =pd.DataFrame(latent_fea)

z_tensor = z.clone().detach().requires_grad_(True)
z = z_tensor.detach().numpy()
z_0 = pd.DataFrame(z)

## coalescence

In [ ]:
train_1 = train_balanced[train_balanced.iloc[: , -1] == 1]
train_1_X = train_1.iloc[:, 0:-1]

In [ ]:
generatedata1 = databuild.DataBuilder(train_1_X)
generateloader1 = databuild.DataLoader(dataset = generatedata1, batch_size = 219)

with torch.no_grad():

  for batch_idx, data in enumerate(generateloader1):
    data = data.to(device)
    optimizer.zero_grad()
    mu, logvar, latent_fea, z, generated_data, latent_label = model(data)

input_data_tensor = torch.tensor(train_1_X.values.astype(np.float32)) 

sigma = torch.exp(logvar/2)

# sample z from q
no_samples = n_samples
q = torch.distributions.Normal(mu.mean(axis=0), sigma.mean(axis=0))
noise = gaussian(4, n_samples)
z = q.rsample(sample_shape=torch.Size([no_samples])) + 0.5 * torch.tensor(noise).float()

generated_data = generate_fake(model, z)
cols = ["chamber_size","top","bottom","dt"]
df_fake_1 = pd.DataFrame(generated_data, columns = cols) 

latent_fea_tensor = latent_fea.clone().detach().requires_grad_(True)
latent_fea = latent_fea_tensor.detach().numpy()
latent_fea_1 =pd.DataFrame(latent_fea)

z_tensor = z.clone().detach().requires_grad_(True)
z = z_tensor.detach().numpy()
z_1 = pd.DataFrame(z)

## Concat

In [ ]:
df_fake_0 = df_fake_0.iloc[:, 0:-1]
df_fake_0.insert(4, "Coalescence", 1.0)
df_fake_1 = df_fake_1.iloc[:, 0:-1]
df_fake_1.insert(4, "Coalescence", 0.0)

In [ ]:
df_fake = pd.concat([df_fake_0, df_fake_1], axis = 0)

In [ ]:
df_fake.to_csv(f"/content/drive/MyDrive/20221108/dataset/fake_{project}.csv", index=False)